# SSOTAP Service Test Notebook

Automated tests for the Solar System Object TAP (SSOTAP) service.

## Imports

In [ ]:
from lsst.rsp import get_tap_service

## Service Instantiation

In [ ]:
service = get_tap_service("ssotap")
assert service is not None
print(f"SSOTAP service URL: {service.baseurl}")

## Check VOSI Tables

In [ ]:
tables = service.tables
print(f"Found {len(tables)} tables")

## Schema Discovery - List Schemas

In [ ]:
results = service.search("SELECT schema_name, description FROM TAP_SCHEMA.schemas")
assert len(results) > 0
print(f"Found {len(results)} schemas")
results.to_table()

## Schema Discovery - List Tables

In [ ]:
results = service.search("SELECT table_name, description FROM TAP_SCHEMA.tables WHERE schema_name = 'dp03_catalogs_10yr'")
assert len(results) > 0
print(f"Found {len(results)} tables in dp03_catalogs_10yr")
results.to_table()

## Schema Discovery - SSObject Columns

In [ ]:
results = service.search(
    "SELECT column_name, datatype, unit FROM TAP_SCHEMA.columns "
    "WHERE table_name = 'dp03_catalogs_10yr.SSObject' "
    "ORDER BY column_name"
)
assert len(results) > 0
print(f"Found {len(results)} columns in SSObject table")

## Synchronous Query - Small Result Set (TOP 10)

In [ ]:
query = """
SELECT TOP 10 ssObjectId, numObs, arc
FROM dp03_catalogs_10yr.SSObject
"""
results = service.search(query)
assert len(results) <= 10
print(f"Retrieved {len(results)} SSObjects")
results.to_table()

## Synchronous Query - Large Result Set (TOP 10000)

In [ ]:
query = """
SELECT TOP 10000 *
FROM dp03_catalogs_10yr.SSObject
"""
results = service.search(query)
assert len(results) <= 10000
print(f"Retrieved {len(results)} SSObjects")

## Synchronous Query - Filter by Number of Observations

In [ ]:
query = """
SELECT TOP 50 ssObjectId, numObs, arc
FROM dp03_catalogs_10yr.SSObject
WHERE numObs > 100
"""
results = service.search(query)
print(f"Retrieved {len(results)} SSObjects with >100 observations")
results.to_table()

## Query MPCORB Table

In [ ]:
query = """
SELECT TOP 20 ssObjectId, mpcDesignation, e, q, incl, node
FROM dp03_catalogs_10yr.MPCORB
"""
results = service.search(query)
assert len(results) > 0
print(f"Retrieved {len(results)} MPCORB records")
results.to_table()

## Query DiaSource - Cone Search

In [ ]:
query = """
SELECT TOP 50 diaSourceId, ssObjectId, ra, dec, midpointMjdTai
FROM dp03_catalogs_10yr.DiaSource
WHERE CONTAINS(POINT('ICRS', ra, dec),
               CIRCLE('ICRS', 1.0, 0.0, 0.1)) = 1
"""
results = service.search(query)
print(f"Retrieved {len(results)} DiaSources in cone")
if len(results) > 0:
    results.to_table()

## Table Join - MPCORB and SSObject

In [ ]:
query = """
SELECT TOP 30 mpc.ssObjectId, mpc.mpcDesignation, mpc.e, mpc.q,
       sso.numObs, sso.arc
FROM dp03_catalogs_10yr.MPCORB AS mpc
JOIN dp03_catalogs_10yr.SSObject AS sso ON mpc.ssObjectId = sso.ssObjectId
"""
results = service.search(query)
print(f"Retrieved {len(results)} joined records")
results.to_table()

## Async Query - Submit Job

In [ ]:
query = """
SELECT TOP 10 ssObjectId, numObs, arc, g_H, r_H, i_H, z_H, y_H
FROM dp03_catalogs_10yr.SSObject
WHERE numObs > 50
"""
job = service.submit_job(query)
print(f"Job submitted with ID: {job.job_id}")
job.run()
print(f"Job started, phase: {job.phase}")

## Async Query - Wait for Completion

In [ ]:
job.wait(phases=['COMPLETED', 'ERROR'])
print(f"Job phase: {job.phase}")
if job.phase == 'ERROR':
    job.raise_if_error()
assert job.phase == 'COMPLETED', f"Job failed with phase: {job.phase}"

## Async Query - Fetch Results

In [ ]:
results = job.fetch_result()
table = results.to_table()
print(f"Retrieved {len(table)} SSObjects")
assert len(table) > 0
table[:10]

## Async Query - Cleanup

In [ ]:
job.delete()
print("Job deleted")

## Query SSSource Table

In [ ]:
query = """
SELECT TOP 50 ssObjectId, diaSourceId, heliocentricX, heliocentricY, heliocentricZ,
       topocentricX, topocentricY, topocentricZ
FROM dp03_catalogs_10yr.SSSource
"""
results = service.search(query)
print(f"Retrieved {len(results)} SSSource records")
results.to_table()

## Test Complete

In [ ]:
print("All SSOTAP service tests completed successfully.")